# Chain-of-Verification GPT4 Template
Chain-of-Verification (CoVe) is a prompt engineering technique to bypass hallucinations. An LLM generates a baseline response to a user query, but this might contain errors. CoVe helps by creating a plan comprising of verification questions that are used to validate the information. This process results in more accurate answers than the initial response. The final answer is revised based on these validations. **[ Link to Paper](https://arxiv.org/pdf/2309.11495.pdf)**

**How to use template?**
1. Download `cove_template_config.json` [here](https://github.com/lastmile-ai/aiconfig/blob/main/cookbooks/Chain-of-Verification/cove_template_config.json).
2. Follow instructions in this notebook.

*Need a playground to test your prompts?*
Go to this [AI Workbook](https://lastmileai.dev/workbooks/clpa6nv2x00fdqp38oefmz29c) to experiment with your prompts/model params.

In [1]:
# # Install AIConfig package
# !pip install python-aiconfig

In [2]:
# Import required modules from AIConfig and other dependencies
import openai
import json
import pandas as pd
from aiconfig import AIConfigRuntime, CallbackManager, InferenceOptions

# #Set the OpenAI key. The following code is specific for Google Colab
# #We recommend using environment variables for setting the open ai key
# from google.colab import userdata
# openai.api_key = userdata.get('OPENAI_API_KEY')
openai.api_key = open("/home/jacobjensen/secrets/openai_api_key.txt", "r").read().strip()

/home/jacobjensen/.pyenv/versions/aiconfig/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/jacobjensen/.pyenv/versions/aiconfig/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Make sure to upload `cove_template_config.json`to Files in Colab Notebook
file_path = "cove_template_config.json"
config = AIConfigRuntime.load(file_path) # loads config
config.callback_manager = CallbackManager([])

inference_options = InferenceOptions() # setup streaming

## 1. Baseline Response
Prompt LLM with user question. The baseline response from the LLM might contain inaccuracies that we will want to verify.

**Prompt: Name 15 politicians who were born in New York City, New York.**

In [4]:
# TODO: Update baseline_prompt but ensure it is structured in a way that outputs a list of entities where each can be verified.
baseline_prompt = "Name 15 politicians who were born in New York City, New York."
# baseline_prompt = "List 20 programming languages that were developed in the United States. Include the developer name in parentheses."

# Run baseline prompt to generate initial response which might contain errors
async def run_baseline_prompt(baseline_prompt):
    config.update_parameter("baseline_prompt", baseline_prompt)
    config.save()

    await config.run("baseline_response_gen", options=inference_options) # run baseline prompt
    return config.get_output_text("baseline_response_gen")

baseline_response = await run_baseline_prompt(baseline_prompt)

1. Theodore Roosevelt - 26th President of the United States
2. Franklin D. Roosevelt - 32nd President of the United States
3. Donald Trump - 45th President of the United States
4. Michael Bloomberg - Former Mayor of New York City
5. Rudy Giuliani - Former Mayor of New York City
6. Alexander Hamilton - Founding Father of the United States
7. Eliot Spitzer - Former Governor of New York
8. Chuck Schumer - U.S. Senator from New York
9. Kirsten Gillibrand - U.S. Senator from New York
10. Bill de Blasio - Former Mayor of New York City
11. Andrew Cuomo - Former Governor of New York
12. Eric Schneiderman - Former Attorney General of New York
13. Letitia James - Attorney General of New York
14. David Dinkins - Former Mayor of New York City
15. Adam Clayton Powell Jr. - Former U.S. Representative from New York.


## 2. Setup and Test Verification Question
Given both query and baseline response, generate a verification
question that could help to self-analyze if there are any mistakes in the original response. We will use one verification question here.

**Verification Prompt: Where was {{entity}} born?**

In [5]:
# TODO: Update verification question that takes in entity as a parameter
verification_question = "Where was {{entity}} born?"
# verification_question =  "Where was this coding language developed: {{entity}}?"

# Run verification on a single entity from baseline response to test
async def run_single_verification(verification_question, entity):
    params = {"entity": entity}
    config.update_parameter("verification_question", verification_question)
    config.save()

    verification_completion = await config.run("verification", params, options=inference_options)
    return verification_completion

# TODO: Update with an entity from the baseline response
verification_completion = await run_single_verification(verification_question, "AL Sharpton")

Al Sharpton was born in Brooklyn, New York, United States.

## 3. Execute Verifications
Answer each verification question for each entity from the the baseline response. Save the verification results in a single string.

In [6]:
# Extracts entity names from a given baseline response by processing each line with regex.
# TODO: Update regex if the format of the baseline response changes. (ex. not a numbered list)
def gen_entities_list(baseline_response):
  rows = baseline_response.split('\n')
  entities = []

  for row in rows:
      if not row.strip():
          continue
      entities.append(pd.Series(row).str.extract(r'(\d+\.\s)([^,]*)')[1].values[0])

  return entities

# Run verification question for each entity and concatenates returned verifications into a single string.
async def gen_verification_results(entities):
  verification_data = ""
  for n in entities:
      params = {
          "verification_question": verification_question,
          "entity": n
      }
      verification_completion = await config.run("verification", params, options=inference_options)
      single_verification_text = config.get_output_text("verification")
      verification_data += " " + single_verification_text
      print("\n")

  return verification_data


entities = gen_entities_list(baseline_response)
verification_data = await gen_verification_results(entities)

Theodore Roosevelt was born in New York City, New York, United States.

Franklin D. Roosevelt was born in Hyde Park, New York, United States.

Donald Trump was born in Queens, New York, United States.

Michael Bloomberg was born in Boston, Massachusetts, United States.

Rudy Giuliani was born in Brooklyn, New York, United States.

Alexander Hamilton was born in Charlestown, Nevis, in the British West Indies.

Eliot Spitzer was born in The Bronx, New York, United States.

Chuck Schumer was born in Brooklyn, New York, United States.

Kirsten Gillibrand was born in Albany, New York, United States.

Bill de Blasio was born in Manhattan, New York, United States.

Andrew Cuomo was born in Queens, New York, United States.

Eric Schneiderman was born in New York City, New York, United States.

Letitia James was born in Brooklyn, New York, United States.

David Dinkins was born in Trenton, New Jersey, United States.

Adam Clayton Powell Jr. was born in New Haven, Connecticut, United States.



## 4. Generate Revised Response
Given the discovered inconsistencies (if any), generate a revised response incorporating the verification results.

In [7]:
# Generated the revised response using verification data
params = {"verification_results": verification_data}
revised_response = await config.run("final_response_gen", params, options=inference_options)

### Revised Response 
1. Theodore Roosevelt - 26th President of the United States
2. Donald Trump - 45th President of the United States
3. Rudy Giuliani - Former Mayor of New York City
4. Eliot Spitzer - Former Governor of New York
5. Chuck Schumer - U.S. Senator from New York
6. Bill de Blasio - Former Mayor of New York City
7. Andrew Cuomo - Former Governor of New York
8. Eric Schneiderman - Former Attorney General of New York
9. Letitia James - Attorney General of New York

### Failed Entities 
1. Franklin D. Roosevelt - He was born in Hyde Park, New York, not New York City.
2. Michael Bloomberg - He was born in Boston, Massachusetts, not New York City.
3. Alexander Hamilton - He was born in Charlestown, Nevis, in the British West Indies, not New York City.
4. Kirsten Gillibrand - She was born in Albany, New York, not New York City.
5. David Dinkins - He was born in Trenton, New Jersey, not New York City.
6. Adam Clayton Powell Jr. - He was born in New Haven, Connecticut, not New Yo

In [8]:
config.__dict__

{'name': 'Chain-of-Verification (CoVe)  Template',
 'schema_version': 'latest',
 'metadata': ConfigMetadata(parameters={'baseline_prompt': 'Name 15 politicians who were born in New York City, New York.', 'verification_question': 'Where was {{entity}} born?'}, models={'gpt-4': {'model': 'gpt-4', 'top_p': 1, 'temperature': 0, 'presence_penalty': 0, 'frequency_penalty': 0}}, default_model=None, model_parsers=None),
 'description': '',
 'prompts': [Prompt(name='baseline_response_gen', input='{{baseline_prompt}}', metadata=PromptMetadata(model=ModelMetadata(name='gpt-4', settings={'system_prompt': ''}), tags=None, parameters={}, remember_chat_context=False), outputs=[ExecuteResult(output_type='execute_result', execution_count=0, data={'content': '1. Theodore Roosevelt - 26th President of the United States\n2. Franklin D. Roosevelt - 32nd President of the United States\n3. Donald Trump - 45th President of the United States\n4. Michael Bloomberg - Former Mayor of New York City\n5. Rudy Giulia

In [9]:
len(config.prompts[1].outputs)

17

In [11]:
config

AIConfigRuntime(name='Chain-of-Verification (CoVe)  Template', schema_version='latest', metadata=ConfigMetadata(parameters={'baseline_prompt': 'Name 15 politicians who were born in New York City, New York.', 'verification_question': 'Where was {{entity}} born?'}, models={'gpt-4': {'model': 'gpt-4', 'top_p': 1, 'temperature': 0, 'presence_penalty': 0, 'frequency_penalty': 0}}, default_model=None, model_parsers=None), description='', prompts=[Prompt(name='baseline_response_gen', input='{{baseline_prompt}}', metadata=PromptMetadata(model=ModelMetadata(name='gpt-4', settings={'system_prompt': ''}), tags=None, parameters={}, remember_chat_context=False), outputs=[ExecuteResult(output_type='execute_result', execution_count=0, data={'content': '1. Theodore Roosevelt - 26th President of the United States\n2. Franklin D. Roosevelt - 32nd President of the United States\n3. Donald Trump - 45th President of the United States\n4. Michael Bloomberg - Former Mayor of New York City\n5. Rudy Giuliani -

In [10]:
config.save()